In [ ]:
! mkdir ../data
! wget https://storage.googleapis.com/usc-data/newsworthiness-project/final-matching-articles-and-meetings.csv
! wget https://storage.googleapis.com/usc-data/newsworthiness-project/full-meeting-data-with-headers.jsonl.zip
! wget https://storage.googleapis.com/usc-data/newsworthiness-project/full_newsworthiness_training_data.jsonl
! wget https://storage.googleapis.com/usc-data/newsworthiness-project/sfchron-fetched-articles.jsonl.zip

In [67]:
# unzip these files and put them in the ../data directory
# ... 

# Reading in City Council Minutes

First, we read in the transcribed city city council meeting minutes. These were all obtained by downloading videos from these pages: https://sanfrancisco.granicus.com/player/clip/43243?view_id=10&meta_id=992444&redirect=true&h=b9650faa96d53d07034d556d92f18771

In [ ]:
import os 
import glob
import pandas as pd 
city_council_data_df = pd.read_json('../data/full_newsworthiness_training_data.jsonl',  lines=True)

These city council minutes have a `label` column that is `True` or `False`. This label column indicates whether or not the city council policy has been written about in news articles.

In [48]:
city_council_data_df.head()

,clip_id,index,class_name,text,time,proposal_number,end_time,header,transcribed_text,label,date
0,16637,18,agenda0,130031 Pursuant to Charter Sections 2.103 and ...,633.004,130031,635.705,COMMUNICATION,"[{'text': ' Colleagues, Madam Clerk, why don't...",False,2013-01-15
1,16637,24,agenda0,"121007 Ordinance authorizing, pursuant to Char...",903.883,121007,918.880,CONSENT AGENDA,"[{'text': 'Madam Clerk, could you please call ...",False,2013-01-15
2,16637,25,agenda0,121064 Ordinance amending the San Francisco Bu...,918.880,121064,942.348,CONSENT AGENDA,"[{'text': ' Thank you.', 'speaker': 'SPEAKER_2...",False,2013-01-15
3,16637,26,agenda0,121139 Resolution approving and authorizing th...,942.348,121139,959.000,CONSENT AGENDA,[{'text': 'This has been kind of a longstandin...,False,2013-01-15
4,16637,41,agenda0,"120997 Ordinance appropriating $843,000 of Sta...",959.000,120997,959.000,CONSENT AGENDA,"[{'text': 'Last year alone, I'm sorry, since 2...",False,2013-01-15


The transcribed text of the city council meeting looks like this, with time-stamps and a speaker ID (these are randomly assigned, but used to track the speaker across the entire course of the meeting.)

In [50]:
city_council_data_df['transcribed_text'].iloc[1]

[{'text': 'Madam Clerk, could you please call item 12?',
  'speaker': 'SPEAKER_29',
  'start': 902.02,
  'end': 903.883},
 {'text': ' Item 12 is an ordinance appropriating $843,000 of state reserves and approximately $1.4 million from school districts set aside funds for the San Francisco Unified School District for fiscal year 2012 through 2013.',
  'speaker': 'SPEAKER_28',
  'start': 903.883,
  'end': 915.267},
 {'text': 'Supervisor Kim.',
  'speaker': 'SPEAKER_28',
  'start': 915.267,
  'end': 918.44},
 {'text': ' Thank you.',
  'speaker': 'SPEAKER_24',
  'start': 918.44,
  'end': 918.88},
 {'text': 'I realize that we are now finally coming to near end on discussion around the supplemental appropriation, and I just want to take a moment to thank my co-sponsors, Supervisors Campos, Marr, and Avalos, and also to Supervisors Cohen and Chu for your support for this supplemental.',
  'speaker': 'SPEAKER_24',
  'start': 918.88,
  'end': 937.206}]

# Meetings and Articles

Now, let's go deeper into the data we did in the last project, which shows the news articles along with the city council proposal text:

In [72]:
final_matching_df = pd.read_csv('../data/final-matching-articles-and-meetings.csv', index_col=0)

In [56]:
# this is a snipped of the news article that covered the city council minute

print(final_matching_df['summary_text'].iloc[0])

On Tuesday, the San Francisco Board of Supervisors will for the second time this summer consider amendments to the Airbnb law regulating short-term rentals to fix the mess former Supervisor David Chiu's industry-sponsored bill created last year. The Board of Supervisors has an opportunity to find a compromise to show that a city can fully embrace home sharing and also restrict landlords from operating illegal hotels. Six months of reports and analyses from government agencies, the media, Airbnb and an academic study provide the data that the Board of Supervisors needs to support an ordinance that facilitates legal short-term renting while giving government agencies tools for proactively regulating away the bad actors. The Board of Supervisors needs to find a reasonable compromise on home sharing so that we can get back to confronting San Francisco's larger housing challenge.

On Tuesday, the San Francisco Board of Supervisors will for the second time this summer consider amendments to 

In [55]:
# this is the proposal text

final_matching_df['meeting text'].iloc[0]

"Administrative Code - Short-Term Residential Rentals. Ordinance amending the Administrative Code to revise the Residential Unit Conversion Ordinance to: revise the definition of interested parties who may enforce the provisions of Chapter 41A, through a private right of action to include permanent residents residing within 100 feet of the residential unit; create an additional private right of action under certain circumstances; change the administrative hearing process from mandatory to at the request of any party found in violation of this Chapter; create an Office of Short-Term Residential Rental Administration and Enforcement staffed by the Planning Department, Department of Building Inspection, and Tax Collector's Office; and affirming the Planning Department's determination under the California Environmental Quality Act."

In [86]:
# these are how many city council minutes were covered

city_council_data_df['label'].value_counts()

label
False    15799
True      1301
Name: count, dtype: int64

# Additional information (if needed)

Here is some more information about both the meeting and the news articles if you need them.

In [88]:
# full article information/ text
# the original final_matching_df doesn't have the full article text, 
# so you might want to look at the actual text

json_file = '../data/sfchron-fetched-articles.jsonl'
articles = []
import json
for line in open(json_file):
    articles.append(json.loads(line))

sf_articles_df = pd.DataFrame(articles)

final_matching_df['key'] = (final_matching_df['article_url']
     .str.split(')')
     .str.get(-1)
     .str.replace('https://', 'http://')
     .str.replace('www.', '')
     .str.replace('http://sfchronicle.com', '')
)

In [89]:
matching_df_with_full_text = (
    sf_articles_df
         .assign(key=lambda df: df['article_url'].str.split('sfchronicle.com').str.get(-1))
         [['key', 'article_text']]
         .merge(final_matching_df, on='key')
)

matching_df_with_full_text.head()

,key,article_text,city,committee,meeting date,File #,meeting text,summary_text,article_url,article_publish_date
0,/bayarea/article/Botanical-Garden-fee-isn-t-ke...,Stopping to smell the roses and other exotic p...,San Francisco,Board of Supervisors,2013-07-23,130548,Park Code - Various Fees. Ordinance amending t...,"On Tuesday, the Board of Supervisors will deci...",http://www.sfchronicle.com/bayarea/article/Bot...,2013-08-21 14:00:46+00:00
1,/bayarea/article/Botanical-Garden-fee-isn-t-ke...,Stopping to smell the roses and other exotic p...,San Francisco,Board of Supervisors,2013-07-23,130537,Lease and Management Agreement - Botanical Gar...,"On Tuesday, the Board of Supervisors will deci...",http://www.sfchronicle.com/bayarea/article/Bot...,2013-08-21 14:00:46+00:00
2,/bayarea/matier-ross/article/Push-for-private-...,Sheriff Ross Mirkarimi acknowledges last week ...,San Francisco,Board of Supervisors,2013-12-10,131084,Appropriation - General Hospital Improvement G...,San Francisco General Hospital officials tried...,http://www.sfchronicle.com/bayarea/matier-ross...,2013-11-13 02:51:50+00:00
3,/bayarea/matier-ross/article/Push-for-private-...,Sheriff Ross Mirkarimi acknowledges last week ...,San Francisco,Board of Supervisors,2013-10-29,130922,"Collective Bargaining Agreement, Amendment No....",San Francisco General Hospital officials tried...,http://www.sfchronicle.com/bayarea/matier-ross...,2013-11-13 02:51:50+00:00
4,/bayarea/article/S-F-General-Sheriff-s-Dept-bl...,San Francisco General Hospital nurses failed t...,San Francisco,Board of Supervisors,2014-03-04,131229,Contract - Sheriff's Department - Secure Reent...,Hospital officials told the investigators that...,http://www.sfchronicle.com/bayarea/article/S-F...,2014-02-02 01:58:27+00:00


In [74]:
## full meeting information

full_meeting_data_with_headers =  pd.read_json('../data/full-meeting-data-with-headers.jsonl', lines=True)
full_meeting_data_with_headers.head(2)

,clip_id,index,class_name,text,time,proposal_number,end_time,header,transcribed_text,speakers
0,16593,0,agenda0,1 ROLL CALL AND PLEDGE OF ALLEGIANCE,122.0,NaN,123.0,ROLL CALL,[{'text': 'The first is a communication from t...,[SPEAKER_27]
1,16593,1,agenda0,2 AGENDA CHANGES,123.0,NaN,123.0,AGENDA CHANGE,[{'text': 'The first is a communication from t...,[SPEAKER_27]
